# Projet Optimisation - Partie 2
---

## Imports

In [2]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize) # To print more elements of an array in terminal
import pandas as pd
from scipy.optimize import linprog
import matplotlib.pyplot as plt

# For modelising more complex problems
import cvxpy as cp

## Parameters

## Datas

In [5]:
# Loads dataframe
sites_df = pd.read_csv('Data-partie-2/Sites.csv')
offshore_df = pd.read_csv('Data-partie-2/Rendements_offshore.csv', header=None).to_numpy()
onshore_df = pd.read_csv('Data-partie-2/Rendements_onshore.csv', header=None).to_numpy()

# Get index of onshore and offshore sites and convert to numpy array
offshore_idx = sites_df[sites_df['capacite offshore'] == 'Oui']['index site'].to_numpy(dtype=int)
onshore_idx = sites_df[sites_df['capacite offshore'] == 'Non']['index site'].to_numpy(dtype=int)

# Get number of sites (n) and number of hours in a year (m)
n = sites_df.shape[0]
m = offshore_df.shape[1]

# Get max capacity of offshore and onshore sites
max_capacity = np.zeros(n)
max_capacity[offshore_idx] = sites_df[sites_df['capacite offshore'] == 'Oui']['capacites'].to_numpy(dtype=float)
max_capacity[onshore_idx] = sites_df[sites_df['capacite offshore'] == 'Non']['capacites'].to_numpy(dtype=float)

# Filter offshore and onshore dataframes
offshore_df = offshore_df[offshore_idx, :]
onshore_df = onshore_df[onshore_idx, :]
rendements_on_off = np.zeros((n, m)) # Matrix that contains all the rendements of onshore and offshore sites at the correct indices
rendements_on_off[offshore_idx, :] = offshore_df
rendements_on_off[onshore_idx, :] = onshore_df

# Loads consumption and hydro production
consumptions = pd.read_csv('Data-partie-2/Consommations.csv', header=None).to_numpy()
hydro_productions = pd.read_csv('Data-partie-2/Apports-hydro.csv', header=None).to_numpy()

# Get number of country (p)
p = consumptions.shape[0]

# Costs
onshore_install_cost = 168903 # [euro/MWh/year]
offshore_install_cost = 300336 # [euro/MWh/year]

gaz_install_cost = 94956 # [euro/MW/year]
gaz_production_cost = 65 # [euro/MWh]

# Country max stock per country [MWh]
max_stock = np.array([0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6])

# Max turbine and pump power per country [MW] 
max_turbine_power = np.array([8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101])
max_pump_power = np.array([5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ])

# Turbine efficiency
turbine_efficiency = 0.85

15


## Question 4

blabla

---